# Human Ankle Joint Manifold Representation

In [ ]:
# Rotation axis in a unitary vector and angle in degrees
def rot_ax_ang(u,alpha):
    na=(1-cos(alpha))
    ca=cos(alpha)
    sa=sin(alpha)
    #rotation matrix
    R=matrix([[u[0]^2*na+ca, u[0]*u[1]*na-u[2]*sa, u[0]*u[2]*na+u[1]*sa],
            [u[0]*u[1]*na+u[2]*sa, u[1]^2*na+ca, u[1]*u[2]*na-u[0]*sa],
            [u[2]*u[0]*na-u[1]*sa, u[2]*u[1]*na+u[0]*sa, u[2]^2*na+ca]])
    return R

#finding the twist unitary vectors 
w1x,w1y,w1z = var('w_1_x,w_1_y,w_1_z')
w2x,w2y,w2z = var('w_2_x,w_2_y,w_2_z')
r1x,r1y,r1z = var('r_1_x,r_1_y,r_1_z')
r2x,r2y,r2z = var('r_2_x,r_2_y,r_2_z')
w1=vector([w1x,w1y,w1z])
w2=vector([w2x,w2y,w2z])
r1=vector([r1x,r1y,r1z])
r2=vector([r2x,r2y,r2z])
#the perpendicular component
v1=-w1.cross_product(r1)
v2=-w2.cross_product(r2)

#initial point
pm0x,pm0y,pm0z = var('pm_0_x,pm_0_y,pm_0_z')
PM0=vector([pm0x,pm0y,pm0z])
#angles for the talocrural and subtalar rotations
theta1=var('theta_1')
theta2=var('theta_2')

#six dimensional vector xi mapping 
xi1=matrix([v1[0],v1[1],v1[2],w1[0],w1[1],w1[2]]).transpose()
xi2=matrix([v2[0],v2[1],v2[2],w2[0],w2[1],w2[2]]).transpose()

#transformation of exponential matrix of rotation
Rexp1=rot_ax_ang(w1,theta1)

#rotation matrix component of the homogeneous transformation
vexp1=(matrix.identity(3)-Rexp1)*(w1.cross_product(v1))

#conformation of the homogeneous transformation matrix
MTH1=(Rexp1.augment(vexp1)).stack(vector([0,0,0,1]))

#components for the subtalar axis
Rexp2=rot_ax_ang(w2,theta2)
vexp2=(matrix.identity(3)-Rexp2)*(w2.cross_product(v2))
MTH2=(Rexp2.augment(vexp2)).stack(vector([0,0,0,1]))

#initial identtity matrix
gst0PM=matrix([[1,0,0,PM0[0]],[0,1,0,PM0[1]],[0,0,1,PM0[2]],[0,0,0,1]])

#product of exponentials, replace the fitting values
MTHPM=MTH1*MTH2*gst0PM
om1=vector([-0.750, -0.280, 0.600])
om2=vector([-0.0890, -0.950, 0.310])
r1=vector([-1.74, 0.000, -54.3])
r2=vector([-0.562, 0.000, -60.8])
pm0=vector([0.000629, 0.000272, -126])
poe=MTHPM.subs(r_1_x=r1[0],r_1_y=r1[1],r_1_z=r1[2],r_2_x=r2[0],r_2_y=r2[1],r_2_z=r2[2])
poe=poe.subs(w_1_x=om1[0],w_1_y=om1[1],w_1_z=om1[2])
poe=poe.subs(w_2_x=om2[0],w_2_y=om2[1],w_2_z=om2[2])
poe=poe.subs(pm_0_x=pm0[0],pm_0_y=pm0[1],pm_0_z=pm0[2])

#arrows
PO=vector([0,0,0])
sc=30 #scale factor
ar1=arrow3d(PO,r1,sc,color='cyan')
ar2=arrow3d(PO,r2,sc,color='magenta')
aw1=arrow3d(r1,r1+sc*om1,sc,color='purple')
aw2=arrow3d(r2,r2+sc*om2,sc,color='violet')
#plot arrows
plarw=ar1+ar2+aw1+aw2

#orthogonal direction vectors
f_spm=vector([poe[0][0],poe[1][0],poe[2][0]])
f_npm=vector([poe[0][1],poe[1][1],poe[2][1]])
f_apm=vector([poe[0][2],poe[1][2],poe[2][2]])

#orientation frame at PM0    
spm=vector([f_spm[0].subs(theta1==0,theta2==0),
            f_spm[1].subs(theta1==0,theta2==0),
            f_spm[2].subs(theta1==0,theta2==0)]).n()
npm=vector([f_npm[0].subs(theta1==0,theta2==0),
           f_npm[1].subs(theta1==0,theta2==0),
           f_npm[2].subs(theta1==0,theta2==0)]).n()
apm=vector([f_apm[0].subs(theta1==0,theta2==0),
           f_apm[1].subs(theta1==0,theta2==0),
           f_apm[2].subs(theta1==0,theta2==0)])
spmv=arrow3d(pm0, pm0 + sc*spm,sc, color ='red')
npmv=arrow3d(pm0, pm0 + sc*npm,sc, color ='green')
apmv=arrow3d(pm0, pm0 + sc*apm,sc, color ='blue')

#plot frame orientation
plfro=spmv+npmv+apmv

#components of the group of rigid movements for the central point
fx=poe[0][3].simplify_full()
fy=poe[1][3].simplify_full()
fz=poe[2][3].simplify_full()
fp=vector([fx,fy,fz])
st1,st2,ct1,ct2= var('st_1,st_2,ct_1,ct_2')
#vectorial parametric function
coords = ((theta1, 0, 2*pi), (theta2,0, 2*pi))
anklemanifold = ParametrizedSurface3D((fx, fy, fz), coords, 'ankle manifold')
amf=anklemanifold.plot(aspect_ratio='automatic', color = 'green', opacity=0.3)
geodesic = anklemanifold.geodesics_numerical([pi/10,pi/10],[1.0,1.0],[0,pi/6,20])
times, points, tangent_vectors, ext_points = zip(*geodesic)
pts=list(points)
epts=list(ext_points)

#geodesic
gdp=line3d(list(ext_points), color = 'red', thickness='10')

am = Manifold(2, 'S^2', latex_name=r'\mathbb{S}^2', start_index=0)
amc.<th1,th2> = am.chart(r'th1:[0,pi/6]:\th1 th2:[-pi/6,pi/6]:periodic:\th2')
E3.<X,Y,Z> = EuclideanSpace()
cartesian = E3.cartesian_coordinates()
Phi = am.diff_map(E3, (fx.subs(theta1==th1,theta2==th2), 
                        fy.subs(theta1==th1,theta2==th2), 
                        fz.subs(theta1==th1,theta2==th2)),
                   name='Phi', latex_name=r'\Phi')
#chart
chart1= amc.plot(chart=cartesian, mapping=Phi, number_values=18, 
           color={th1: 'red', th2: 'green'})
#plot chart, manifold, and geodesic
mfdpm=amf+plarw+plfro
chtgd=chart1+gdp+plarw+plfro
mfdpm.show(viewer='jmol', aspect_ratio= [1,1,1], frame_aspect_ratio= 'automatic', zoom=1)
chtgd.show(viewer='jmol', aspect_ratio= [1,1,1], frame_aspect_ratio= 'automatic', zoom=1)

#save manifold
mfdpm.save_image('mfdpm.png',figsize=[25,25],viewer='threejs',
                  aspect_ratio= [1,1,1], frame_aspect_ratio= 'automatic', zoom=1)
#save chart				  
chtgd.save_image('chgdp.png',figsize=[25,25],viewer='threejs',
                  aspect_ratio= [1,1,1], frame_aspect_ratio= 'automatic', zoom=1)